# Comparisons between two portfolios - example

This is an example of a comparative analysis between 2 portfolios.

We start by importing **azapy** and other useful packages.

In [1]:
import sys
sys.path.append("..")
import azapy as az

print(f"azapy version {az.version()}")

azapy version 1.2.3


### Load historical market data for 2 different portfolios

The 2 sets of portfolio components are listed in `symb1` and `symb2`, respectively.

>Make sure that `mktdir` holds a convenient location to save the market data.
>You can inhibit the saving mechanism by setting `save=False` in the call of `az.readMkT` function (_see `readMkT` documentation_ https://azapy.readthedocs.io/en/latest/)

In [2]:
symb1 = ['GLD', 'TLT', 'XLV', 'VGT', 'VHT']
symb2 = ['PFG', 'QQQ', 'SPY', 'XLF', 'ONEQ']

sdate = "2012-01-01"
edate = "2021-07-27"

mktdir = "../MkTdata"

mktdata1 = az.readMkT(symb1, sdate=sdate, edate=edate, file_dir=mktdir) 
mktdata2 = az.readMkT(symb2, sdate=sdate, edate=edate, file_dir=mktdir)  

read GLD data from file
read TLT data from file
read XLV data from file
read VGT data from file
read VHT data from file

Request between 2012-01-03 : 2021-07-27
                    GLD         TLT         XLV         VGT         VHT
source            yahoo       yahoo       yahoo       yahoo       yahoo
force             False       False       False       False       False
save               True        True        True        True        True
file_dir     ../MkTdata  ../MkTdata  ../MkTdata  ../MkTdata  ../MkTdata
file_format         csv         csv         csv         csv         csv
api_key            None        None        None        None        None
nrow               2407        2407        2407        2407        2407
sdate        2012-01-03  2012-01-03  2012-01-03  2012-01-03  2012-01-03
edate        2021-07-27  2021-07-27  2021-07-27  2021-07-27  2021-07-27
error                No          No          No          No          No
extraction time 0.168 s
read PFG data from file

### Check the historical market data quality for fist portfolio

We use the **azapy** function `summary_MktData`. It returns:

- `symbol` the symbols included in `mktdata1`. These are the portfolio components, 
- `begin` is the beginning of the historical time-series for each symbol,
- `end` is the end date of the historical time series (the most recent date),
- `length` is history length in number of business days.
- `na_total` is the total number of missing values, `nan`,
- `na_b` is the total number of missing records at the beginning of the time-series (in number of business days),
- `na_e` is the total number of missing records at the end of the time-series (in number of business days),
- `cont` is the total number of missing records (at the beginning, end and in the middle of the time-series). 

We are looking for `na_total`, `na_b`, `na_e` and `count` to be `0`. Missing data could be a serious problem. In general gaps in the market data should be filled before proceeding further. 

*For more information see the documentation for `summary_MkTData`.*

In [3]:
az.summary_MkTdata(mktdata1)

,symbol,begin,end,length,na_total,na_b,na_e,cont
0,GLD,2012-01-03,2021-07-27,2407,0,0,0,0
1,TLT,2012-01-03,2021-07-27,2407,0,0,0,0
2,VGT,2012-01-03,2021-07-27,2407,0,0,0,0
3,VHT,2012-01-03,2021-07-27,2407,0,0,0,0
4,XLV,2012-01-03,2021-07-27,2407,0,0,0,0


### Check the historical market data quality for second portfolio

Same as before.

In [4]:
az.summary_MkTdata(mktdata2)

,symbol,begin,end,length,na_total,na_b,na_e,cont
0,ONEQ,2012-01-03,2021-07-27,2407,0,0,0,0
1,PFG,2012-01-03,2021-07-27,2407,0,0,0,0
2,QQQ,2012-01-03,2021-07-27,2407,0,0,0,0
3,SPY,2012-01-03,2021-07-27,2407,0,0,0,0
4,XLF,2012-01-03,2021-07-27,2407,0,0,0,0


### Set the dispersion measure parameters.

In our case we have chosen mCVaR (mixture of 3 CVaR's) dispersion measure. A similar workflow holds for any other dispersion measure.

In [5]:
alpha = [0.95, 0.90, 0.85]
coef = [0.2, 0.3, 0.5]

### Setup the first portfolio 

Two steps:

1. Construct the portfolio class. In our case this is `az.Port_CVaR`.
2. Set the model and trigger the computation of the portfolio time-series
   (in this case rtype='Sharpe' and mu0=0 - the default values).

*For more information see the documentation for `Port_CVaR` class*.

In [6]:
pr1 = az.Port_CVaR(mktdata1, pname='Port1')
port1 = pr1.set_model(alpha=alpha, coef=coef)

### Setup the second portfolio

Same as above.

In [7]:
pr2 = az.Port_CVaR(mktdata2, pname='Port2')
port2 = pr2.set_model(alpha=alpha, coef=coef)

### Setup a `Port_Simple` class to compare the 2 portfolios

The `Port_Simple` class is set from the list of portfolios time-series. 

>Note the must call to `set_model` method.

>Observation: `Port_Simple` is the class that supports the back testing of "Buy and Hold" portfolio (_see its documentation_).
It also can be used as a tool to compare the performance of multiple portfolios. Here we use it in this latter capacity.

In [8]:
pp = az.Port_Simple([port1, port2])
_ = pp.set_model()

### Visualized the 2 portfolios time-series

> Note the use of flag `fancy=True`. It designates the use of *plotly* package for interactive time-series. 

As expected, `Port2` is more aggressive than `Port1`.

In [9]:
_ = pp.port_view_all(componly=True, fancy=True, title="Reltive performance")

### Compare the performances of the two portfolios

- `RR` is the average annual portfolio rate of returns.
- `DD` is the maximum drawdown rate.
- `Beta` is the ratio `RR/DD`.
- `DD_date` is the date of maximum drawdown.
- `DD_start` is the date when the maximum drawdown event had started.
- `DD_end` is the date when the maximum drawdown event had ended. If it is `NaN` then the drawdown is still in progress (`DD` and `DD_date` are only provisional).

> Note the use of flag `fancy=True` - the rates are reported in percent with 2 decimals.

The rate of returns for `Port2` is almost double than for `Port1`. However, the maximum drawdown is 50% larger for `Port2` than for `Port1`.  

In [10]:
pp.port_perf(componly=True, fancy=True)

,RR,DD,RoMaD,DD_date,DD_start,DD_end,DD_days
symbol,,,,,,,
Port2,22.06,-28.56,0.772392,2020-03-16,2020-02-19,2020-05-18,89
Port1,12.84,-18.53,0.692856,2020-03-20,2020-02-19,2020-04-24,65


### Compare the portfolios annual returns 

Except for 2018, `Port2` has significantly higher returns than `Port1`.
This report can be detailed further by asking for the monthly returns. In this case it is worth to investigate separately the performance of the two portfolios.

In [11]:
pp.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,Port1,Port2
year,,
2015,-5.32%,-1.81%
2016,1.68%,6.61%
2017,25.49%,31.32%
2018,6.16%,6.49%
2019,34.15%,32.11%
2020,18.64%,50.66%
2021,2.16%,15.40%


### First portfolio performance (details)

Note that the portfolio drawdown is smaller than any individual component drawdown. Thus, `Port1` has a good diversification for a moderate average return.

In [12]:
pr1.port_perf(fancy=True)

,RR,DD,RoMaD,DD_date,DD_start,DD_end,DD_days
symbol,,,,,,,
Port1,12.84,-18.53,0.692856,2020-03-20,2020-02-19,2020-04-24,65
VGT,23.10,-31.84,0.725512,2020-03-23,2020-02-19,2020-06-09,111
VHT,17.41,-28.85,0.603515,2020-03-23,2020-02-19,2020-06-08,110
XLV,16.74,-28.40,0.589180,2020-03-23,2020-01-22,2020-07-15,175
TLT,4.89,-21.34,0.229103,2021-03-18,2020-08-04,NaN,357
GLD,0.81,-42.11,0.019280,2015-12-17,2012-10-04,2020-07-22,2848


### First portfolio drawdown (details)

Subsequent drawdowns are smaller but still above 10%.

In [13]:
pr1.port_drawdown(fancy=True)

,DD,Date,Start,End,NrDays
No,,,,,
1,-18.53,2020-03-20,2020-02-19,2020-04-24,65
2,-16.71,2015-09-28,2015-08-05,2016-07-06,336
3,-15.55,2018-12-24,2018-08-31,2019-02-05,158
4,-10.73,2021-03-08,2020-08-06,2021-06-10,308
5,-9.03,2016-12-01,2016-09-28,2017-04-24,208


### First portfolio rebalancing weights

Note that recently the portfolio is dominated by GLD. From the beginning of 2020 GLD is over 40%.

In [14]:
pr1.get_weights(fancy=True)

,Droll,Dfix,GLD,TLT,XLV,VGT,VHT
0,2015-06-25,2015-06-24,0.00,10.29,89.71,0.00,0.00
1,2015-09-25,2015-09-24,0.00,16.05,55.59,0.00,28.36
2,2015-12-28,2015-12-24,0.00,28.47,32.76,38.77,0.00
3,2016-03-28,2016-03-24,0.00,40.40,38.86,20.74,0.00
4,2016-06-27,2016-06-24,0.00,40.64,34.47,24.89,0.00
5,2016-09-27,2016-09-26,0.00,46.09,32.28,21.63,0.00
6,2016-12-27,2016-12-23,0.00,38.92,0.00,61.08,0.00
7,2017-03-28,2017-03-27,0.00,37.01,0.00,62.99,0.00
8,2017-06-27,2017-06-26,0.00,36.09,0.00,63.91,0.00
9,2017-09-26,2017-09-25,0.00,36.03,0.00,63.97,0.00


### Second portfolio performance (details)

Note that the maximum drawdown of `Port2` is the same as for QQQ, with a rate of returns smaller than QQQ. The `Port2` diversification is not better than for QQQ. Overall `Port2` is less attractive than a buy-and-hold position in QQQ.

In [15]:
pr2.port_perf(fancy=True)

,RR,DD,RoMaD,DD_date,DD_start,DD_end,DD_days
symbol,,,,,,,
Port2,22.06,-28.56,0.772392,2020-03-16,2020-02-19,2020-05-18,89
QQQ,22.66,-28.56,0.793592,2020-03-16,2020-02-19,2020-06-03,105
ONEQ,20.75,-30.16,0.687763,2020-03-23,2020-02-19,2020-06-05,107
SPY,16.04,-33.72,0.475823,2020-03-23,2020-02-19,2020-08-10,173
XLF,15.69,-42.86,0.366010,2020-03-23,2020-02-14,2021-01-06,327
PFG,13.71,-64.73,0.211753,2020-03-23,2018-01-26,2021-05-05,1195


### First portfolio rebalancing weights

As expected with very few exceptions `Port2` is dominated by a single asset, QQQ.

In our opinion `Port2` is an unattractive investment. If the rate of returns and the drawdown levels are acceptable for an investor, then she is better off by investing directly in a buy-and-hold strategy in QQQ.

In [16]:
pr2.get_weights(fancy=True)

,Droll,Dfix,PFG,QQQ,SPY,XLF,ONEQ
0,2015-06-25,2015-06-24,0.00,0.00,87.50,12.50,0.0
1,2015-09-25,2015-09-24,4.24,34.03,0.00,61.74,0.0
2,2015-12-28,2015-12-24,10.29,89.71,0.00,0.00,0.0
3,2016-03-28,2016-03-24,0.00,100.00,0.00,0.00,0.0
4,2016-06-27,2016-06-24,0.00,100.00,0.00,0.00,0.0
5,2016-09-27,2016-09-26,0.00,100.00,0.00,0.00,0.0
6,2016-12-27,2016-12-23,0.00,100.00,0.00,0.00,0.0
7,2017-03-28,2017-03-27,0.00,100.00,0.00,0.00,0.0
8,2017-06-27,2017-06-26,0.00,100.00,0.00,0.00,0.0
9,2017-09-26,2017-09-25,0.00,100.00,0.00,0.00,0.0
